# CF based Recommender System Using Movielens Data
# Haran Arasaratnam
# Date: 06/01/17

I am using the [movielens data set](https://grouplens.org/datasets/movielens/)

Spark MLlib library for Machine Learning provides a Collaborative Filtering implementation by using Alternating Least Squares. The implementation in MLlib has these parameters:

* numBlocks is the number of blocks used to parallelize computation (set to -1 to auto-configure).
* rank is the number of latent factors in the model.
* iterations is the number of iterations to run.
* lambda specifies the regularization parameter in ALS.
* implicitPrefs specifies whether to use the explicit feedback ALS variant or one adapted for implicit feedback data.
* alpha is a parameter applicable to the implicit feedback variant of ALS that governs the baseline confidence in preference observations.



In [1]:
import findspark
findspark.init('/Users/HARAN/spark-2.2.0-bin-hadoop2.7')
import pyspark

from pyspark.sql import SparkSession

In [2]:
spark = SparkSession.builder.appName('rec').getOrCreate()

In [3]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

In [4]:
data = spark.read.csv('movielens_ratings.csv',inferSchema=True,header=True)

In [5]:
data.head()

Row(movieId=2, rating=3.0, userId=0)

In [6]:
data.describe().show()

+-------+------------------+------------------+------------------+
|summary|           movieId|            rating|            userId|
+-------+------------------+------------------+------------------+
|  count|              1501|              1501|              1501|
|   mean| 49.40572951365756|1.7741505662891406|14.383744170552964|
| stddev|28.937034065088994| 1.187276166124803| 8.591040424293272|
|    min|                 0|               1.0|                 0|
|    max|                99|               5.0|                29|
+-------+------------------+------------------+------------------+



In [7]:
# Smaller dataset so we will use 0.8 / 0.2
(training, test) = data.randomSplit([0.8, 0.2])

In [8]:
# Build the recommendation model using ALS on the training data
als = ALS(maxIter=5, regParam=0.01, userCol="userId", itemCol="movieId", ratingCol="rating")
model = als.fit(training)

Now let's see hwo the model performed!

In [9]:
# Evaluate the model by computing the RMSE on the test data
predictions = model.transform(test)

In [10]:
predictions.show()

+-------+------+------+-----------+
|movieId|rating|userId| prediction|
+-------+------+------+-----------+
|     31|   1.0|    26|  0.7586683|
|     31|   1.0|    27| 0.80506504|
|     31|   1.0|    13|  0.5815114|
|     31|   1.0|    19|  3.1463203|
|     31|   3.0|    14|   2.798922|
|     31|   1.0|    18| -1.8639109|
|     85|   5.0|     8|  1.4811509|
|     85|   3.0|    21| -2.0168858|
|     65|   2.0|     3|   3.297383|
|     65|   1.0|    19|  1.0345389|
|     65|   2.0|    15|  0.3490616|
|     53|   1.0|    25|-0.12021742|
|     78|   1.0|    27| 0.96091104|
|     78|   1.0|    12|  1.7824249|
|     81|   5.0|    28|  2.7345228|
|     81|   1.0|    19|  1.7149227|
|     81|   1.0|     7|-0.37856558|
|     81|   1.0|    21| -3.7129617|
|     81|   3.0|    18|0.037592836|
|     28|   1.0|    10|-0.44257155|
+-------+------+------+-----------+
only showing top 20 rows



In [11]:
# evaluate CF usng RMSE (wrt rating)
evaluator = RegressionEvaluator(metricName="rmse", labelCol="rating",predictionCol="prediction")
rmse = evaluator.evaluate(predictions)
print("Root-mean-square error = " + str(rmse))

Root-mean-square error = 1.938634815849985


recommendation to a user in the same way we did with the test data!

In [12]:
single_user = test.filter(test['userId']==11).select(['movieId','userId'])

In [13]:
# User had 10 ratings in the test data set 
# Realistically this should be some sort of hold out set!
single_user.show()

+-------+------+
|movieId|userId|
+-------+------+
|     10|    11|
|     18|    11|
|     21|    11|
|     22|    11|
|     70|    11|
|     72|    11|
|     77|    11|
|     89|    11|
|     94|    11|
+-------+------+



In [14]:
reccomendations = model.transform(single_user)

In [15]:
reccomendations.orderBy('prediction',ascending=False).show()

+-------+------+-----------+
|movieId|userId| prediction|
+-------+------+-----------+
|     94|    11|  3.6303852|
|     77|    11|  2.6352706|
|     22|    11|  2.4089222|
|     72|    11|  2.0475419|
|     21|    11|  1.2920144|
|     18|    11|  1.2080888|
|     10|    11| 0.48241496|
|     70|    11| 0.40234506|
|     89|    11|-0.23762049|
+-------+------+-----------+



# Done !


# Issues

* To improve the RMSE, use a larger data set
* 
